In [211]:
import pandas as pd

In [212]:
#DATA TRIMESTRAL
#Colombia
col_querie = pd.read_excel('Colombia/colombia_region_total.xlsx',header=9)
col_catego = pd.read_excel('Colombia/colombia_region_xeno.xlsx',header=10)

#Ecuador
ecu_querie = pd.read_excel('Ecuador/ecuador_region_total.xlsx',header=9)
ecu_catego = pd.read_excel('Ecuador/ecuador_region_xeno.xlsx',header=10)

#Peru
per_querie = pd.read_excel('Peru/peru_region_total.xlsx',header=9)
per_catego = pd.read_excel('Peru/peru_region_xeno.xlsx',header=10)


In [213]:
#FUNCIONES

def estructura_querie(df_querie, df_catego):

    def limpieza (df):
        df.drop(columns='population',inplace=True)
        df = pd.melt(df,id_vars='weeks',
                    value_vars=df.columns[1:],
                    var_name='Fecha',
                    value_name='Total')
        
        df['Fecha'] = pd.to_datetime(df['Fecha']).dt.date
        df['Fecha'] = [str(x) for x in df['Fecha']]
        df['ID'] = df['weeks'] + "_" + df['Fecha']
        df.sort_values(by='ID')

        return df 
# -------------------------------------------------------------------
    df_querie,df_catego = limpieza(df_querie),limpieza(df_catego)
    
    df = pd.merge(left      = df_querie,
                  right     = df_catego,
                  left_on   = 'ID',
                  right_on  = 'ID',
                  how = 'inner')
    
    df.rename(columns={'Total_x':'Total Menciones','Total_y':'Total Xenofobia','Fecha_y':'Fecha'},inplace=True)
    df[['Region','Pais']] = df['weeks_x'].str.split(',',expand=True)
# -------------------------------------------------------------------
    df['Tasa Xeno Regiones'] = df['Total Xenofobia'] / df['Total Menciones']
    df['Tasa Xeno Regiones'].fillna(0,inplace=True)
# -------------------------------------------------------------------

    df = df[['Pais','Region','Fecha','Total Menciones','Total Xenofobia','Tasa Xeno Regiones']]

    df.sort_values(by=['Pais'],inplace=True)
    df = df[(df['Region']!=(df['Pais'][0].strip())) & (df['Tasa Xeno Regiones'] != 0)]

    df.sort_values(by=['Region','Fecha'],inplace=True)

    return df

In [214]:
#INDICADOR PAIS
colombia = estructura_querie(col_querie,col_catego)
ecuador = estructura_querie(ecu_querie,ecu_catego)
peru = estructura_querie(per_querie,per_catego)

In [6]:
#EXPORTAR
with pd.ExcelWriter('Tasa Xenofobia Regiones-2023.xlsx') as writer:
    
    colombia.to_excel(writer,sheet_name='Colombia')
    ecuador.to_excel(writer,sheet_name='Ecuador')
    peru.to_excel(writer,sheet_name='Peru')
